In [1]:
!pip install nibabel
!pip install seaborn
!pip install openpyxl

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Convolution3D,Conv3D,Add,concatenate, MaxPool3D,add, Convolution2D,BatchNormalization, GlobalAveragePooling3D, ZeroPadding3D
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPool3D, Flatten, Dense, Dropout, concatenate, Lambda, Layer
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML
from tensorflow.keras import layers

from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPool3D, Flatten, Dense, Dropout, concatenate, Lambda, Layer
from tensorflow.keras.models import Model
import tensorflow as tf

2024-08-20 13:15:05.203068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 13:15:05.203195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 13:15:05.347162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### LUNA Load

In [3]:
luna_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)

(557,)


In [4]:
luna_semi_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/unlabeled_data_Luna.xlsx')
#column_name = 'malignancy'
col_scan='Series Uid'

luna_semi_scan = luna_semi_df[col_scan].tolist()

scans1=[]
for i in luna_semi_scan:
    scans1.append(i)

luna_semi=np.array(scans1)
print(len(luna_semi))

612


In [5]:
datas_luna_semi=[]
scans_luna_semi=[]

for i in luna_semi:
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",i +"*.nii"))

    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_luna_semi.append(data)
        scans_luna_semi.append(i)

X_luna_semi = np.array(datas_luna_semi)
z_luna_semi= np.array(scans_luna_semi)

print(X_luna_semi.shape)

(611, 32, 32, 32)


### Fixed Test

In [6]:
# Path to the Excel file
file_path = '/kaggle/input/semi-testtrainval/test_train_vai_set.xlsx'

# Load the first column from the Excel sheet
luna_fixed_test = pd.read_excel(file_path, usecols=[0])

# Remove NaN values
luna_fixed_test = luna_fixed_test.dropna()

# Display the cleaned DataFrame to verify
print(luna_fixed_test)


                                   luna fixed test set
0    1.3.6.1.4.1.14519.5.2.1.6279.6001.202283133206...
1    1.3.6.1.4.1.14519.5.2.1.6279.6001.200725988589...
2    1.3.6.1.4.1.14519.5.2.1.6279.6001.321935195060...
3    1.3.6.1.4.1.14519.5.2.1.6279.6001.245248446973...
4    1.3.6.1.4.1.14519.5.2.1.6279.6001.222052723822...
..                                                 ...
107  1.3.6.1.4.1.14519.5.2.1.6279.6001.275755514659...
108  1.3.6.1.4.1.14519.5.2.1.6279.6001.877026508860...
109  1.3.6.1.4.1.14519.5.2.1.6279.6001.187966156856...
110  1.3.6.1.4.1.14519.5.2.1.6279.6001.131939324905...
111  1.3.6.1.4.1.14519.5.2.1.6279.6001.204802250386...

[112 rows x 1 columns]


In [7]:
len(luna_fixed_test)

112

In [8]:
datas_luna_fixed_test=[]
labels_luna_fixed_test=[]
scans_luna_fixed_test = []

for i in luna_fixed_test.iloc[:, 0]:  # Assuming i is a value in the first column
    matching_rows = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy']
    
    if not matching_rows.empty:
        lab = matching_rows.values[0]
        img_files = glob.glob(os.path.join(
            "/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",
            i + "*.nii"
        ))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_luna_fixed_test.append(data)
            labels_luna_fixed_test.append(lab)
            scans_luna_fixed_test.append(i)
    else:
        print(f"No matching Series Uid found for: {i}")
X_luna_fixed_test = np.array(datas_luna_fixed_test)
y_luna_fixed_test = np.array(labels_luna_fixed_test)
z_luna_fixed_test= np.array(scans_luna_fixed_test)

print(X_luna_fixed_test.shape)

(112, 32, 32, 32)


### 1st train

In [9]:
best_model1 = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU1.h5')
best_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss1, accuracy1 = best_model1.evaluate(X_luna_fixed_test, y_luna_fixed_test)   

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8924 - loss: 0.3350

I0000 00:00:1724159734.062296     117 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 578ms/step - accuracy: 0.8854 - loss: 0.3438


In [10]:


correct_predictions1 = []
incorrect_predictions1 = []

predictions = best_model1.predict(X_luna_semi)
for i in range(len(X_luna_semi)):
    if predictions[i] > 0.9:
        correct_predictions1.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions1.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions1.append(z_luna_semi[i])

print(len(incorrect_predictions1),len(correct_predictions1))
        
# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions1 = pd.DataFrame(correct_predictions1, columns=['scan', 'malignancy'])

scan_identifiers_z_train2 = np.array([item[0] for item in correct_predictions1])

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 121ms/step
202 409


In [11]:
df_correct_predictions1 = pd.DataFrame(correct_predictions1, columns=['scan', 'malignancy'])

### 2nd Train

In [12]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions1:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi2 = np.array(datas_semi)
z_semi2 = np.array(scans_semi)

print(X_semi2.shape)


(202, 32, 32, 32)


In [13]:
best_model2 = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU2.h5')
best_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss2, accuracy2 = best_model2.evaluate(X_luna_fixed_test, y_luna_fixed_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 270ms/step - accuracy: 0.8368 - loss: 0.5326


In [14]:
correct_predictions2 = []
incorrect_predictions2 = []

predictions = best_model2.predict(X_semi2)
for i in range(len(X_semi2)):
    if predictions[i] > 0.9:
        correct_predictions2.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions2.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions2.append(z_luna_semi[i])

print(len(incorrect_predictions2),len(correct_predictions2))


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 339ms/step
79 123


In [15]:
df_correct_predictions2 = pd.DataFrame(correct_predictions2, columns=['scan', 'malignancy'])

### 3rd train

In [16]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions2:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi3 = np.array(datas_semi)
z_semi3 = np.array(scans_semi)

print(X_semi3.shape)

(79, 32, 32, 32)


In [17]:
best_model3 = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU3.h5')
best_model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss3, accuracy3 = best_model3.evaluate(X_luna_fixed_test, y_luna_fixed_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 260ms/step - accuracy: 0.8679 - loss: 0.3386


In [18]:
correct_predictions3 = []
incorrect_predictions3 = []

predictions = best_model3.predict(X_semi3)
for i in range(len(X_semi3)):
    if predictions[i] > 0.9:
        correct_predictions3.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions3.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions3.append(z_luna_semi[i])

print(len(incorrect_predictions3),len(correct_predictions3))
df_correct_predictions3 = pd.DataFrame(correct_predictions3, columns=['scan', 'malignancy'])

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
30 49


### 4th

In [19]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions3:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi4 = np.array(datas_semi)
z_semi4 = np.array(scans_semi)

print(X_semi4.shape)

(30, 32, 32, 32)


In [20]:
best_model4 = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU4.h5')
best_model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss4, accuracy4 = best_model4.evaluate(X_luna_fixed_test, y_luna_fixed_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 258ms/step - accuracy: 0.8418 - loss: 0.6057


In [21]:
correct_predictions4 = []
incorrect_predictions4 = []

predictions = best_model4.predict(X_semi4)
for i in range(len(X_semi4)):
    if predictions[i] > 0.9:
        correct_predictions4.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions4.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions4.append(z_luna_semi[i])

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions4 = pd.DataFrame(correct_predictions4, columns=['scan', 'malignancy'])

print(df_correct_predictions4.shape)
print(len(incorrect_predictions4),len(correct_predictions4))

scan_identifiers_z_train5 = np.array([item[0] for item in correct_predictions4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
(21, 2)
9 21


### 5th

In [22]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions4:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi5 = np.array(datas_semi)
z_semi5 = np.array(scans_semi)

print(X_semi5.shape)

(9, 32, 32, 32)


In [23]:
best_model5 = tf.keras.models.load_model('/kaggle/input/semi-testtrainval/semiSupervised_HU5.h5')
best_model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss5, accuracy5 = best_model5.evaluate(X_luna_fixed_test, y_luna_fixed_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 277ms/step - accuracy: 0.8560 - loss: 0.4192


In [24]:
predictions = best_model5.predict(X_semi5)

correct_predictions5 = []
incorrect_predictions5 = []

for i in range(len(X_semi5)):
    if predictions[i] > 0.9:
        correct_predictions5.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions5.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions5.append(z_luna_semi[i])

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions5 = pd.DataFrame(correct_predictions5, columns=['scan', 'malignancy'])

print(len(incorrect_predictions5), len(correct_predictions5))

scan_identifiers_z_train6 = np.array([item[0] for item in correct_predictions5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
6 3


In [25]:
df_combined = pd.concat(
    [
        df_correct_predictions1,
        df_correct_predictions2,
        df_correct_predictions3,
        df_correct_predictions4,
        df_correct_predictions5
    ],
    ignore_index=True
)

# Specify the file path for the Excel file
file_path = '/kaggle/working/combined_correct_predictions.xlsx'

# Write the combined DataFrame to a single Excel sheet
df_combined.to_excel(file_path, index=False)

print(f"Excel file saved as {file_path}")


Excel file saved as /kaggle/working/combined_correct_predictions.xlsx
